<h1>Getting started with pyspark</h1>
<p>Spark is a distributed in memory platform intended for large scale data analysis.  There are two basic classes that you interact with in Spark: the Spark Context and RDD's which are Resilient Distributed Datasets.  Let's start by verifying that you've set thing up correctly by calling 'sc', which should output something like:</p>

>`<pyspark.context.SparkContext at 0x1094d6ed0>`

<p>If this doesn't work, you muddled the settup, and need to figure that out before you can move forward.</p>

In [1]:
sc

<h2>Get some data</h2>
<p>Spark is made for looking at data....lots of data.  Since we're running locally, we're restricted to the memory of our laptops and only a few cores.  As such we'll use small data as an illustrative example from sklearn.</p>

In [2]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))

#The clips are long, here is just a sample of the first entry
print dataset.data[0][:50] 

print len(dataset.data)

Well i'm not sure about the story nad it did seem 
11314


<h2>SHUT UP AND START THE SPARK!</h2>
<p>Gotcha.  Let''s start by creating our first rdd.  To do so, we use the
`sc.parralllelize` command</p>

In [3]:
rdd = sc.parallelize(dataset.data)
print type(rdd)

<class 'pyspark.rdd.RDD'>


<h2>Now for the word count</h2>
<p>The word count is performed in 3 steps, and can actually be written by one line of code:
* tokenize the clips into words - flatMap
* clean the words, put them into tuples as pairs of (key, 1)
* combine tuples by their key

At that point, you'd have a collection of ('word', count) tuples.  For the purpose of demonstration, we also sort to find the most frequent words, and return a small collection.  Since we don't filter stop words, we expect words like 'the', 'is', and 'a' to dominate the list.

In [4]:
terms = rdd.flatMap(lambda text: text.split())\
            .map(lambda word: (word.strip(".,-;?").lower(), 1))\
            .reduceByKey(lambda a, b: a+b)\
            .sortBy(lambda x: x[1], ascending=False)\
            .take(10)
            
print terms

[(u'the', 104952), (u'to', 52420), (u'of', 46651), (u'a', 42348), (u'and', 41621), (u'in', 30324), (u'is', 29958), (u'i', 27792), (u'that', 26700), (u'it', 20382)]
